In [1]:
import numpy as np

from qiskit import QuantumCircuit
from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit.library import EfficientSU2

from qiskit_addon_cutting.instructions import Move
from qiskit_addon_cutting import partition_problem
from qiskit_addon_cutting import generate_cutting_experiments

In [2]:
def get_subexperiment_circuits_for_gate_cutting(num_reps):
    qc = EfficientSU2(4, entanglement="linear", reps=num_reps).decompose()
    qc.assign_parameters([0.4] * len(qc.parameters), inplace=True)
    
    observable = SparsePauliOp(["ZZII", "IZZI", "-IIZZ", "XIXI", "ZIZZ", "IXIX"])
    
    partitioned_problem = partition_problem(
        circuit=qc, partition_labels="AABB", observables=observable.paulis
    )

    subcircuits = partitioned_problem.subcircuits
    subobservables = partitioned_problem.subobservables
    bases = partitioned_problem.bases
    
    subexperiments, coefficients = generate_cutting_experiments(
        circuits=subcircuits, observables=subobservables, num_samples=np.inf
    )
    
    return subexperiments, coefficients, bases

In [87]:
for i in range(1, 3):
    subexperiments, coefficients, bases = get_subexperiment_circuits_for_gate_cutting(i)
    
    print(len(subexperiments["A"]))

12
72


In [28]:
for basis_map in bases[0].maps:
    print(basis_map[0])
    print(basis_map[1])
    print()

[Instruction(name='sdg', num_qubits=1, num_clbits=0, params=[])]
[Instruction(name='h', num_qubits=1, num_clbits=0, params=[]), Instruction(name='sdg', num_qubits=1, num_clbits=0, params=[]), Instruction(name='h', num_qubits=1, num_clbits=0, params=[])]

[Instruction(name='s', num_qubits=1, num_clbits=0, params=[])]
[Instruction(name='h', num_qubits=1, num_clbits=0, params=[]), Instruction(name='s', num_qubits=1, num_clbits=0, params=[]), Instruction(name='h', num_qubits=1, num_clbits=0, params=[])]

[Instruction(name='sdg', num_qubits=1, num_clbits=0, params=[]), Instruction(name='qpd_measure', num_qubits=1, num_clbits=0, params=[])]
[]

[Instruction(name='sdg', num_qubits=1, num_clbits=0, params=[]), Instruction(name='qpd_measure', num_qubits=1, num_clbits=0, params=[])]
[Instruction(name='h', num_qubits=1, num_clbits=0, params=[]), Instruction(name='z', num_qubits=1, num_clbits=0, params=[]), Instruction(name='h', num_qubits=1, num_clbits=0, params=[])]

[]
[Instruction(name='h', nu

In [89]:
index = 20
subexperiments["A"][index].draw()

┌─────────┐┌─────────┐     ┌─────────┐┌─────────┐»
                    q22_0: ┤ Ry(0.4) ├┤ Rz(0.4) ├──■──┤ Ry(0.4) ├┤ Rz(0.4) ├»
                           ├─────────┤├─────────┤┌─┴─┐└──┬───┬──┘├─────────┤»
                    q22_1: ┤ Ry(0.4) ├┤ Rz(0.4) ├┤ X ├───┤ S ├───┤ Ry(0.4) ├»
                           └─────────┘└─────────┘└───┘   └───┘   └─────────┘»
observable_measurements: 2/═════════════════════════════════════════════════»
                                                                            »
       qpd_measurements: 1/═════════════════════════════════════════════════»
                                                                            »
«                                           ┌─────────┐┌─────────┐┌─┐   
«                    q22_0: ─────────────■──┤ Ry(0.4) ├┤ Rz(0.4) ├┤M├───
«                           ┌─────────┐┌─┴─┐├─────────┤├─────────┤└╥┘┌─┐
«                    q22_1: ┤ Rz(0.4) ├┤ X ├┤ Ry(0.4) ├┤ Rz(0.4) ├─╫─┤M├
«                           └─────────┘└───┘└─────────┘└─────────┘ ║ └╥┘
«observable_measurements: 2/═══════════════════════════════════════╩══╩═
«                                                                  0  1 
«       qpd_measurements: 1/════════════════════════════════════════════
«

In [92]:
subexperiments["B"][22].draw()

┌─────────┐┌─────────┐┌───┐┌───┐┌───┐     »
                    q23_0: ┤ Ry(0.4) ├┤ Rz(0.4) ├┤ H ├┤ S ├┤ H ├──■──»
                           ├─────────┤├─────────┤└───┘└───┘└───┘┌─┴─┐»
                    q23_1: ┤ Ry(0.4) ├┤ Rz(0.4) ├───────────────┤ X ├»
                           └─────────┘└─────────┘               └───┘»
observable_measurements: 2/══════════════════════════════════════════»
                                                                     »
       qpd_measurements: 1/══════════════════════════════════════════»
                                                                     »
«                           ┌─────────┐┌─────────┐┌───┐┌─────┐┌─┐┌───┐     »
«                    q23_0: ┤ Ry(0.4) ├┤ Rz(0.4) ├┤ H ├┤ Sdg ├┤M├┤ H ├──■──»
«                           ├─────────┤├─────────┤└───┘└─────┘└╥┘└───┘┌─┴─┐»
«                    q23_1: ┤ Ry(0.4) ├┤ Rz(0.4) ├─────────────╫──────┤ X ├»
«                           └─────────┘└─────────┘             ║      └───┘»
«observable_measurements: 2/═══════════════════════════════════╬═══════════»
«                                                              ║           »
«       qpd_measurements: 1/═══════════════════════════════════╩═══════════»
«                                                              0           »
«                           ┌─────────┐┌─────────┐┌─┐   
«                    q23_0: ┤ Ry(0.4) ├┤ Rz(0.4) ├┤M├───
«                           ├─────────┤├─────────┤└╥┘┌─┐
«                    q23_1: ┤ Ry(0.4) ├┤ Rz(0.4) ├─╫─┤M├
«                           └─────────┘└─────────┘ ║ └╥┘
«observable_measurements: 2/═══════════════════════╩══╩═
«                                                  0  1 
«       qpd_measurements: 1/════════════════════════════
«